# Skyview Factor computation

In this notebook, we generate raster tiles for sky view factor and topographic correction factor as reported by this papers :
- Helbig, N., et al. “Parameterizing Surface Wind Speed over Complex Topography.” 2017

In [1]:
import geopandas as gpd
import numpy as np
import rasterio
from scipy import signal

In [2]:
dem = rasterio.open('C:/Users/PaulBillecocq/CodeRepositories/UdS_Code/violaine_GNP_subgrid/ancillary_data/cdem_dem_082N_EPSG32611_100x100_CLIPPED.tif')
hrdps_cells = gpd.read_file('C:/Users/PaulBillecocq/CodeRepositories/UdS_Code/violaine_GNP_subgrid/ancillary_data/hrdps_selected_cells.gpkg')
dem_array = dem.read(1)

## Part 1 : Sky View Factor Computation

### Mean squared slope

$\mu = \frac{\overline{[(z_{i,j} - z_{i+1,j})^2 + (z_{i,j} - z_{i,j+1})^2]}^{1/2}}{\Delta x\sqrt{2}}$

We will be using two convolution kernels to compute the two coordinates related sums. Those arrays will then be squared separately, then summed, averaged, and sqrted

In [3]:
def compute_mean_squared_slope(input_array, delta_x):
    '''
    Computes the mean squared slope according to Helbig, N., and Löwe, H. (2014), 
    Parameterization of the spatially averaged sky view factor in complex topography, J. Geophys. Res. Atmos., 119, 4616– 4625, doi:10.1002/2013JD020892.
    
    Parameters:
    input_array (np.array): array of elevations values on which to compute mean squared slope
    delta_x (float): cell resolution of the dem array in meters
    
    Returns:
    mu (float): mean squared slope value
    '''
    kernel_1 = np.array([
        [0, 0, 0],
        [0, 1, 0],
        [0, -1, 0]
    ])

    kernel_2 = np.array([
        [0, 0, 0],
        [0, 1, -1],
        [0, 0 , 0]
    ])

    # Compute convolution of both kernels on the dem array. Reduce their size to ensure that they are same size and that each cell is corresponding and that each cell was computed without using padded data
    member_1 = signal.convolve2d(input_array, kernel_1, mode='same')[1:,:-1]
    member_2 = signal.convolve2d(input_array, kernel_2, mode='same')[:-1,1:]
    mu = np.sqrt(np.mean(np.square(member_1) + np.square(member_2))) / (delta_x * np.sqrt(2))
    
    return mu

### Typical width of topographic features

$\xi = \frac{\sigma_{DEM}\sqrt{2}}{\mu}$

In [4]:
def compute_typical_width(mu, elevation_standard_deviation):
    '''
    Computes the typical width of topographic features (xi) in the provided DEM array, according to Helbig, N., and Löwe, H. (2014), 
    Parameterization of the spatially averaged sky view factor in complex topography, J. Geophys. Res. Atmos., 119, 4616– 4625, doi:10.1002/2013JD020892.
    
    Parameters:
    mu (float): mean squared slope value for the corresponding dem_array
    elevation_standard_deviation (float): standard deviation of elevation values on the desired DEM
    
    Returns:
    xi (float): typical width of topographic features in the given dem array (in meters)
    '''
    xi = elevation_standard_deviation * np.sqrt(2) / mu
    
    return xi

### Sky View Factor

$F_{sky}(L/\xi, \mu) = \Big ( 1 - \Big (1 - \frac{1}{(1 + a \mu^{b})^{c}} \Big ) e^{-d (L/\xi)^{-2}} \Big )$

with a = 3.354688, b = 1.998767, c = 0.20286, and d = 5.951

For more details, see : Helbig, N., and Löwe, H. (2014), Parameterization of the spatially averaged sky view factor in complex topography, J. Geophys. Res. Atmos., 119, 4616– 4625, doi:10.1002/2013JD020892.
    

In [5]:
def compute_sky_view_factor(mu, xi, L,):
    '''
    Computes the Sky View Factor as stated above
    
    Parameters:
    mu (float): mean squared slope compute for the unresolved fine-scale topography
    xi (float): typical width of topographic features in the given dem array (in meters)
    L (float): cell resolution of the coarse resolution Numerical Weather Prediction cell (in meters)
    
    Returns:
    F_sky (float): sky view factor paramter
    '''
    # Hard coded parameters, optimized by Helbig and Löwe in the 2014 paper.
    a = 3.354688
    b = 1.998767
    c = 0.20286
    d = 5.951
    
    F_sky = (1 - (1 - 1 / ((1 + a * mu ** b) ** c)) * np.exp(-d * (L / xi) ** (-2)))
    
    return F_sky

### Compute Sky View Factor
Basically, we need a value for $\mu$ and $\xi$ for each set of DEM pixels describing the geometry of each HRDPS' cell underlying topography. But there is one problem : HRDPS grid and DEM grid are not corresponding (HRDPS raw is not even on a plane ...).

The idea is to compute the centroid of each HRDPS cell polygon, get it's reference pixel in the DEM array, and extract a 2.5 km x 2.5 km square around it. Compute geometric parameters, and populate the information in each row of the HRDPS grid geodataframe.

In [6]:
# DEM cell resolution in meters
delta_x = 100
# HRDPS cell resolution meters
L = 2500

In [16]:
def sky_view_factor_processing_wrapper(dem_raster, gdf_row):
    '''
    This function is meant to be used in the apply framework of a pandas DataFrame to generate sky view factor parameters for each cell
    
    Parameters:
    dem_raster (rasterio object): rasterio object corresponding to the dem
    gdf_row (gpd row): row of the apply function
    '''
    dem_array = dem_raster.read(1)
    centroid = gdf_row['geometry'].centroid
    try:
        array_coords = dem_raster.index(centroid.x, centroid.y)
        # Build a 12 x 12 array centered around the centroid (e.g. 2.5 km x 2.5 km)
        array_subset = dem_array[(array_coords[0] - 12):(array_coords[0] + 13), (array_coords[1] - 12):(array_coords[1] + 13)]
        mu = compute_mean_squared_slope(array_subset, delta_x)
        xi = compute_typical_width(mu, np.std(dem_array))
        F_sky = compute_sky_view_factor(mu, xi, L)
        # Populate row with computed parameters
        gdf_row['mu'] = mu
        gdf_row['xi'] = xi
        gdf_row['F_sky'] = F_sky

        return gdf_row
    except:
        station = gdf_row['Station']
        print(f'Exception processing cell {station}')
        gdf_row['mu'] = np.nan
        gdf_row['xi'] = np.nan
        gdf_row['F_sky'] = np.nan
        
        return gdf_row

In [17]:
hrdps_cells = hrdps_cells.apply(lambda x : sky_view_factor_processing_wrapper(dem, x), axis=1)

#### Save hrdps shapefile with Sky View Factor parameters as attributes

In [18]:
hrdps_cells.set_geometry('geometry', crs='EPSG:32611', inplace=True)
hrdps_cells.dropna(inplace=True)
hrdps_cells.to_file('C:/Users/PaulBillecocq/CodeRepositories/UdS_Code/violaine_GNP_subgrid/ancillary_data/hrdps_grid_with_downscalingVW_topo_params.shp')

## Part 2 : VW Downscaling

### Elevation Laplacian

$\nabla^{2} z = \nabla^{2}z' \frac{\Delta_x}{4}$

$\nabla^{2} z' = \Big (z(x - \Delta x, y) + z(x + \Delta x, y) + z(x, y - \Delta x) + z(x, y + \Delta x) - 4z(x, y) \Big ) \big / \Delta x^{2}$

Again, we will be using a convolution kernel

In [19]:
def compute_elevation_laplacien(input_array, delta_x):
    '''
    Computes the discrete elevation Laplacian according to formulation above
    
    Parameters:
    input_array (np.array): array of elevations values
    delta_x (float): cell resolution of the dem array in meters
    
    Returns:
    nabla_z (np.array): elevation Laplacian
    '''
    kernel = np.array([
        [0, 1, 0],
        [1, -4, 1],
        [0, 1, 0]
    ])

    # Compute convolution of laplacian kernel on the dem array. Reduce its size to ensure that each cell was computed without using padded data
    nabla_z_array = signal.convolve2d(input_array, kernel, mode='valid') / (delta_x ** 2)
    nabla_array_padded = np.ones(input_array.shape) * np.nan
    nabla_array_padded[1:-1, 1:-1] = nabla_z_array
    nabla_array_padded = nabla_array_padded * delta_x / 4
    
    return nabla_array_padded

### Topographic downscaling factor

$X_{topo}^{dsc}(\nabla^{2}z, \mu) = \Big ( 1 - \frac{a' \nabla^{2}z}{1 + a' |\nabla^{2}z|^{b'}} \Big ) \frac{c'}{1 + d'\mu^{e'}}$

with a′ = 17.0393, b′ = 0.737, c′ = 1.0234, d′ = 0.3794, and e′ = 1.9821

For more details, see Helbig, N., et al. “Parameterizing Surface Wind Speed over Complex Topography.” Journal of Geophysical Research: Atmospheres, vol. 122, no. 2, John Wiley & Sons, Ltd, Jan. 2017, pp. 651–67, doi:10.1002/2016JD025593.

In [20]:
def compute_downscaling_factor(elevation_laplacian, mu):
    '''
    Computes downscaling according to the definition above.
    
    Parameters:
    elevation_laplacian_array (float): elevation laplacian value
    mu (float): mean squared slope value for the overlying HRDPS cell
    
    Returns:
    x_dsc_topo (float): downscaling factor for current pixel
    '''
    # Constant parameters c.f. Helbig et. al., 2017
    a_prime = 17.0393
    b_prime = 0.737
    c_prime = 1.0234
    d_prime = 0.3794
    e_prime = 1.9821
    
    x_dsc_topo = (1 - (a_prime * elevation_laplacian) / (1 + a_prime * np.abs(elevation_laplacian) ** b_prime)) * c_prime / (1 + d_prime + mu ** e_prime)
    
    return x_dsc_topo

compute_downscaling_factor_vectorized = np.vectorize(compute_downscaling_factor)

In [22]:
def subset_array(full_array, center_coordinates):
    '''
    Subsets the input array in 250*250 array centered on the pixel
    '''
    array_subset = full_array[(center_coordinates[0] - 12):(center_coordinates[0] + 13), (center_coordinates[1] - 12):(center_coordinates[1] + 13)]
    
    return array_subset
    
def downscaling_parameters_processing_wrapper(gdf_row, dem_raster, F_array, X_array, nabla_array):
    '''
    This function is meant to be used in the apply framework of a pandas DataFrame to generate sky view factor and downscaling parameter for each DEM pixel
    
    Parameters:
    dem_raster (rasterio object): rasterio object corresponding to the dem
    gdf_row (gpd row): row of the apply function
    '''
    dem_array = dem_raster.read(1)
    centroid = gdf_row['geometry'].centroid
    array_coords = dem_raster.index(centroid.x, centroid.y)
    dem_array_subset = subset_array(dem_array, array_coords)
    nabla_array_subset = subset_array(nabla_array, array_coords)
    
    mu = compute_mean_squared_slope(dem_array_subset, delta_x)
    xi = compute_typical_width(mu, np.std(dem_array))
    try:
        F_sky = compute_sky_view_factor(mu, xi, L)
        X_dsc_topo_subarray = compute_downscaling_factor_vectorized(nabla_array_subset, mu)
        # populate topo params array
        F_array[(array_coords[0] - 12):(array_coords[0] + 13), (array_coords[1] - 12):(array_coords[1] + 13)] = F_sky
        X_array[(array_coords[0] - 12):(array_coords[0] + 13), (array_coords[1] - 12):(array_coords[1] + 13)] = X_dsc_topo_subarray

        return (F_array, X_array)
    except:
        return (-999, -999)

In [23]:
hrdps_cells

,rlat,rlon,altitude,lat,lon,name,geometry,mu,xi,F_sky
0,-2.6725,-1.883720,1514.413696,51.199078,-117.698578,GNP_HRDPS_84,"POLYGON ((449978.392 5670903.494, 452483.892 5...",0.519350,1285.702790,0.974593
1,-2.6725,-1.861213,1605.248169,51.200027,-117.662750,GNP_HRDPS_85,"POLYGON ((452484.741 5670985.712, 454988.537 5...",0.333262,2003.614749,0.998637
2,-2.6725,-1.838722,1864.007812,51.200970,-117.626923,GNP_HRDPS_86,"POLYGON ((454987.688 5671067.774, 457493.178 5...",0.394012,1694.693326,0.994702
3,-2.6725,-1.816215,2069.360107,51.201893,-117.591064,GNP_HRDPS_87,"POLYGON ((457494.027 5671149.905, 459997.813 5...",0.525349,1271.019750,0.973249
4,-2.6725,-1.793723,2080.091553,51.202801,-117.555237,GNP_HRDPS_88,"POLYGON ((459996.964 5671231.882, 462502.444 5...",0.488691,1366.364326,0.980968
...,...,...,...,...,...,...,...,...,...,...
67,-2.5150,-1.793723,1990.847412,51.360172,-117.565414,GNP_HRDPS_195,"POLYGON ((459420.138 5688738.218, 461925.957 5...",0.571465,1168.451822,0.962001
68,-2.5150,-1.771217,1949.391479,51.361080,-117.529434,GNP_HRDPS_196,"POLYGON ((461926.807 5688820.232, 464430.923 5...",0.608689,1096.996810,0.951924
69,-2.5150,-1.748725,1578.116455,51.361977,-117.493484,GNP_HRDPS_197,"POLYGON ((464430.074 5688902.096, 466935.885 5...",0.590363,1131.049681,0.956982
70,-2.5150,-1.726218,1286.756714,51.362854,-117.457489,GNP_HRDPS_198,"POLYGON ((466935.885 5688984.002, 469441.693 5...",0.336930,1981.806319,0.998493


In [24]:
F_array = np.ones(dem_array.shape) * np.nan
X_array = np.ones(dem_array.shape) * np.nan
nabla_array = compute_elevation_laplacien(dem_array, delta_x)

for i in range(len(hrdps_cells)):
    c_row = hrdps_cells.iloc[i]
    print(f'Processing VSTATION {c_row.name} at index {i}')
    F_array, X_array = downscaling_parameters_processing_wrapper(c_row, dem, F_array, X_array, nabla_array)

Processing VSTATION 0 at index 0
Processing VSTATION 1 at index 1
Processing VSTATION 2 at index 2
Processing VSTATION 3 at index 3
Processing VSTATION 4 at index 4
Processing VSTATION 5 at index 5
Processing VSTATION 6 at index 6
Processing VSTATION 7 at index 7
Processing VSTATION 8 at index 8
Processing VSTATION 9 at index 9
Processing VSTATION 10 at index 10
Processing VSTATION 11 at index 11
Processing VSTATION 12 at index 12
Processing VSTATION 13 at index 13
Processing VSTATION 14 at index 14
Processing VSTATION 15 at index 15
Processing VSTATION 16 at index 16
Processing VSTATION 17 at index 17
Processing VSTATION 18 at index 18
Processing VSTATION 19 at index 19
Processing VSTATION 20 at index 20
Processing VSTATION 21 at index 21
Processing VSTATION 22 at index 22
Processing VSTATION 23 at index 23
Processing VSTATION 24 at index 24
Processing VSTATION 25 at index 25
Processing VSTATION 26 at index 26
Processing VSTATION 27 at index 27
Processing VSTATION 28 at index 28
Proce

In [25]:
X_array

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [26]:
 with rasterio.open(
     'C:/Users/PaulBillecocq/CodeRepositories/UdS_Code/violaine_GNP_subgrid/ancillary_data/X_dsc_topo.tif',
     'w',
     driver='GTiff',
     height=X_array.shape[0],
     width=X_array.shape[1],
     count=1,
     dtype=X_array.dtype,
     crs=dem.crs,
     transform=dem.transform,
 ) as dst:
     dst.write(X_array, 1)

In [27]:
F_array

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [29]:
 with rasterio.open(
     'C:/Users/PaulBillecocq/CodeRepositories/UdS_Code/violaine_GNP_subgrid/ancillary_data/skf.tif',
     'w',
     driver='GTiff',
     height=F_array.shape[0],
     width=F_array.shape[1],
     count=1,
     dtype=F_array.dtype,
     crs=dem.crs,
     transform=dem.transform,
 ) as dst:
     dst.write(F_array, 1)